In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 603.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
IMG_SIZE = (128, 128)
NUM_CLASSES = 5
BATCH_SIZE = 32
DATA_DIR = "/content/drive/MyDrive/EDGE AI/df"

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale = 1/.255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

Found 1514 images belonging to 5 classes.


In [ ]:
val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

Found 377 images belonging to 5 classes.


In [ ]:
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model_mobile = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
model_mobile.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
initial_epochs = 10

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(patience=2)
]

In [ ]:
history = model_mobile.fit(
    train_generator,
    validation_data=val_generator,
    epochs=initial_epochs,
    callbacks=callbacks
)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 248s 5s/step - accuracy: 0.5299 - loss: 1.5012 - val_accuracy: 0.6844 - val_loss: 1.1152 - learning_rate: 0.0010
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.8082 - loss: 0.7560 - val_accuracy: 0.6446 - val_loss: 1.1753 - learning_rate: 0.0010
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.8240 - loss: 0.6762 - val_accuracy: 0.6897 - val_loss: 1.0640 - learning_rate: 0.0010
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.8569 - loss: 0.5644 - val_accuracy: 0.6578 - val_loss: 1.3578 - learning_rate: 0.0010
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.8602 - loss: 0.5333 - val_accuracy: 0.6817 - val_loss: 1.2707 - learning_rate: 0.0010
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.8869 - loss: 0.4657 - val_accuracy: 0.7056 - val_loss: 1.2100 - learning_rate: 1.0000e-04


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

In [ ]:
model_mobile_pre = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
model_mobile_pre.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_mobile_pre.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 10
history_finetune = model_mobile_pre.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs,
    callbacks=callbacks
)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.2895 - loss: 1.9955 - val_accuracy: 0.2467 - val_loss: 1.8450 - learning_rate: 1.0000e-05
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.4262 - loss: 1.5376 - val_accuracy: 0.4058 - val_loss: 1.6981 - learning_rate: 1.0000e-05
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.5893 - loss: 1.3135 - val_accuracy: 0.5172 - val_loss: 1.6457 - learning_rate: 1.0000e-05
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.6992 - loss: 1.0971 - val_accuracy: 0.6154 - val_loss: 1.5697 - learning_rate: 1.0000e-05
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.7471 - loss: 0.9642 - val_accuracy: 0.6552 - val_loss: 1.5384 - learning_rate: 1.0000e-05
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.7742 - loss: 0.8846 - val_accuracy: 0.6605 - val_loss: 1.5340 - learning_rate: 1.0000e-05
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.8039 - loss:

In [ ]:
model_mobile.export("mobilenet_saved_model")

Saved artifact at 'mobilenet_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_628')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  134282294138000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414187536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414187728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414184656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414184464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414188880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414189264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414189648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414189456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134280414186960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13428

In [ ]:
model_mobile_pre.export("mobilenet_pre_saved_model")

In [ ]:
model_mobile.save("mobilenet_model.keras")

In [ ]:
model_mobile_pre.save("mobilenet_pre_model.keras")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
img_size = 128
batch_size = 32

In [ ]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=3, verbose=1)

In [ ]:
model_tinycnn = models.Sequential([
    layers.Conv2D(16, (3, 3), padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4), input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model_tinycnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stop = EarlyStopping(
    patience=5,
    restore_best_weights=True,
    monitor='val_loss'
)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    factor=0.2,
    patience=3,
    verbose=1
)

In [ ]:
history = model_tinycnn.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.3658 - loss: 4.7272 - val_accuracy: 0.4695 - val_loss: 1.2719 - learning_rate: 0.0010
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.5877 - loss: 1.0472 - val_accuracy: 0.6313 - val_loss: 1.0867 - learning_rate: 0.0010
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.5978 - loss: 0.9567 - val_accuracy: 0.7109 - val_loss: 1.0171 - learning_rate: 0.0010
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.6657 - loss: 0.8338 - val_accuracy: 0.7135 - val_loss: 1.0097 - learning_rate: 0.0010
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.7153 - loss: 0.7146 - val_accuracy: 0.7454 - val_loss: 0.8602 - learning_rate: 0.0010
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.7449 - loss: 0.6581 - val_accuracy: 0.7878 - val_loss: 0.8841 - learning_rate: 0.0010
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.7342 - loss: 0.6609 - val_accuracy: 

In [ ]:
model_tinycnn.export("tinycnn_saved_model")

Saved artifact at 'tinycnn_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_645')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  134282294136272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294148368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294147216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294136080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294148944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294137040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294134736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282288735888: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
model_tinycnn.save("tinycnn_model.keras")

## ResNet8

In [ ]:
from tensorflow.keras import layers, models, regularizers

In [ ]:
def resnet_block(inputs, filters, kernel_size=3, stride=1):
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same',
                      kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same',
                      kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or inputs.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = inputs

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

In [ ]:
def build_resnet(input_shape=(128, 128, 3), num_classes=5, num_blocks=4):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    filters = 16
    for i in range(num_blocks):
        stride = 2 if i % 2 == 0 and i != 0 else 1
        x = resnet_block(x, filters, stride=stride)
        filters *= 2 if i % 2 == 0 and i != 0 else 1

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
model_resnet8 = build_resnet(num_blocks=4)

In [ ]:
model_resnet8.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

history = model_resnet8.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 521s 11s/step - accuracy: 0.4532 - loss: 1.3064 - val_accuracy: 0.1936 - val_loss: 1.7996 - learning_rate: 0.0010
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.7160 - loss: 0.7530 - val_accuracy: 0.5332 - val_loss: 1.1797 - learning_rate: 0.0010
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7853 - loss: 0.5897 - val_accuracy: 0.6340 - val_loss: 0.9799 - learning_rate: 0.0010
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8081 - loss: 0.5376 - val_accuracy: 0.6101 - val_loss: 0.9049 - learning_rate: 0.0010
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8467 - loss: 0.5069 - val_accuracy: 0.7135 - val_loss: 0.8749 - learning_rate: 0.0010
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8244 - loss: 0.4998 - val_accuracy: 0.7003 - val_loss: 0.7715 - learning_rate: 0.0010
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8423 - loss: 0.4660 - val_accuracy

In [ ]:
model_resnet8.save("resnet8_model.keras")

In [ ]:
model_resnet8.export("resnet8_saved_model")

Saved artifact at 'resnet8_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  138928598229712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138928598230096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138923416313808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138928598229520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138923416315152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138923416313040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138928598223568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138923416313232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138923416314000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138923416313424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13892341631

# ResNet10

In [ ]:
model_resnet10 = build_resnet(num_blocks=5)

In [ ]:
model_resnet10.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

history = model_resnet8.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.8811 - loss: 0.3420 - val_accuracy: 0.7905 - val_loss: 0.6329 - learning_rate: 3.1250e-05
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8837 - loss: 0.3007 - val_accuracy: 0.7931 - val_loss: 0.6621 - learning_rate: 3.1250e-05
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8962 - loss: 0.2971 - val_accuracy: 0.7958 - val_loss: 0.6386 - learning_rate: 3.1250e-05
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.9055 - loss: 0.2934 - val_accuracy: 0.7851 - val_loss: 0.6494 - learning_rate: 1.5625e-05
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8955 - loss: 0.3108 - val_accuracy: 0.7984 - val_loss: 0.6183 - learning_rate: 1.5625e-05
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.9113 - loss: 0.2855 - val_accuracy: 0.8037 - val_loss: 0.6146 - learning_rate: 1.5625e-05
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.8980 - loss:

In [ ]:
model_resnet10.save("resnet10_model.keras")

In [ ]:
model_resnet10.export("resnet10_saved_model")

Saved artifact at 'tinycnn_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_645')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  134282294136272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294148368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294147216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294136080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294148944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294137040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282294134736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134282288735888: TensorSpec(shape=(), dtype=tf.resource, name=None)


# Quantization

## Initiation

In [ ]:
def representative_data_gen():
    for _ in range(100):
        images, _ = next(train_generator)  # Use next() to get a batch
        yield [tf.cast(images, tf.float32)]

## MobileNet

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("mobilenet_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [ ]:
tflite_model = converter.convert()

In [ ]:
with open("mobilenet_int8_quant.tflite", "wb") as f:
    f.write(tflite_model)

## TinyCNN

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("tinycnn_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [ ]:
tflite_model = converter.convert()

In [ ]:
with open("tinycnn_int8_quant.tflite", "wb") as f:
    f.write(tflite_model)

## MobileNet Keras

In [ ]:
model = tf.keras.models.load_model("mobilenet_model.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 74 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [ ]:
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [ ]:
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpfv4edpu9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  135527764773968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764772240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764774352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764774160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764773776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764773200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764776464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764777424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764775888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135527764771088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1355277647785

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
with open("mobilenet_keras_int8.tflite", "wb") as f:
    f.write(tflite_model)

## TinyCNN Keras

In [ ]:
model = tf.keras.models.load_model("tinycnn_model.keras")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [ ]:
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [ ]:
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpam8usg7w'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_8')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  135526155167184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155165840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155166032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155164112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155164880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155163152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155163728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135526155165072: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
with open("tinycnn_keras_int8.tflite", "wb") as f:
    f.write(tflite_model)

## ResNet8

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("resnet8_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [ ]:
tflite_model = converter.convert()

In [ ]:
with open("resnet8_int8_quant.tflite", "wb") as f:
    f.write(tflite_model)

## ResNet8 Keras

In [ ]:
model = tf.keras.models.load_model("resnet8_model.keras")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [ ]:
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [ ]:
tflite_model = converter.convert()

Saved artifact at '/tmp/tmp5xqst6ib'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  138922799035216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138922799035600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138922799034832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921128074640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138922799034064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138922799034448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921128082128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921128073296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921128073680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921128076368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921128081552

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
with open("resnet8_keras_int8.tflite", "wb") as f:
    f.write(tflite_model)